In [1]:
import sys
sys.path.append("src")

In [2]:
from imutils.video import FileVideoStream
from tqdm import tqdm
import pickle
from facial_analysis import *
from torch.utils.data import DataLoader, Dataset

In [3]:
path = "Engagement/FaceEngageDataset/u1/u1_00001.mp4"

In [4]:
video = FileVideoStream(path).start()

batch_size = 128

frame_idx = []
i = 0
results = {"Emotions": [], "Valence": [], "Arousal": []}

def process_batch(faces):
    global results
    with torch.no_grad():
        out = net(faces)
        expr = torch.softmax(out["expression"], dim=-1)
        results["Emotions"].append(expr)
        results["Valence"].append(out["valence"])
        results["Arousal"].append(out["arousal"])
        
faces = []
while True:
    frame = video.read()
    if frame is None:
        break
    face = detect_face(frame)
    if face is not None:
        faces.append(clean_face(face))
        frame_idx.append(i)
        if len(faces) == batch_size:
            process_batch(torch.stack(faces))
            faces = []
    i += 1

if len(faces):
    process_batch(torch.stack(faces))

for k, v in results.items():
    results[k] = np.array(torch.cat(v).cpu())
results["Frame Index"] = frame_idx

In [5]:
len(frame_idx), results["Emotions"].shape, results["Valence"].shape

(264, (264, 8), (264,))